## 라이브러리 로딩

In [252]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

## 데이터 로딩 및 확인

In [253]:
train = pd.read_csv('train.csv') 
test = pd.read_csv('test.csv')

In [254]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      1459 non-null   int64  
 1   hour                    1459 non-null   int64  
 2   hour_bef_temperature    1457 non-null   float64
 3   hour_bef_precipitation  1457 non-null   float64
 4   hour_bef_windspeed      1450 non-null   float64
 5   hour_bef_humidity       1457 non-null   float64
 6   hour_bef_visibility     1457 non-null   float64
 7   hour_bef_ozone          1383 non-null   float64
 8   hour_bef_pm10           1369 non-null   float64
 9   hour_bef_pm2.5          1342 non-null   float64
 10  count                   1459 non-null   float64
dtypes: float64(9), int64(2)
memory usage: 125.5 KB


In [255]:
train.head()

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
3,8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57.0
4,9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431.0


In [256]:
test.head()

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5
0,0,7,20.7,0.0,1.3,62.0,954.0,0.041,44.0,27.0
1,1,17,30.0,0.0,5.4,33.0,1590.0,0.061,49.0,36.0
2,2,13,19.0,1.0,2.1,95.0,193.0,0.020,36.0,28.0
3,4,6,22.5,0.0,2.5,60.0,1185.0,0.027,52.0,38.0
4,5,22,14.6,1.0,3.4,93.0,218.0,0.041,18.0,15.0


## 결측치 확인 및 처리

In [257]:
train.isnull().sum()

id                          0
hour                        0
hour_bef_temperature        2
hour_bef_precipitation      2
hour_bef_windspeed          9
hour_bef_humidity           2
hour_bef_visibility         2
hour_bef_ozone             76
hour_bef_pm10              90
hour_bef_pm2.5            117
count                       0
dtype: int64

In [258]:
test.isnull().sum()

id                         0
hour                       0
hour_bef_temperature       1
hour_bef_precipitation     1
hour_bef_windspeed         1
hour_bef_humidity          1
hour_bef_visibility        1
hour_bef_ozone            35
hour_bef_pm10             37
hour_bef_pm2.5            36
dtype: int64

In [259]:
# train.fillna(0,inplace = True)
# test.fillna(0,inplace = True)
# 중위수로 결측치 처리하기
train_na = train.fillna(train.median())
test_na = test.fillna(test.median())
test = test_na.drop(['id'],axis=1)
train_x = train_na.drop(['id','count'],axis = 1)
train_y = train_na['count']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(train_x,train_y,test_size=0.2,random_state=100)
x_train.shape, y_train.shape
# 데이터 스케일링
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(x_train)
x_train_scaled = ss.transform(x_train)
test_scaled = ss.transform(test)

예측모델의 파이프라인 구축해서 각 모델의 점수를 비교해서 최상의 모델을 선택하고 하이퍼 파라메터적용하기

ploy증가하는 함수도 적용해서 선형이 아닌 비선형도 성능 측정하기

In [260]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression,Ridge,Lasso, SGDRegressor
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor,GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
# 교차 검증
from sklearn.model_selection import cross_validate
# 특성을 추가하여 비 선형으로 만들기
from sklearn.preprocessing import PolynomialFeatures

# 1. 가능한 모든 모델로 측정한다.
temp1  = [
    'KNeighborsRegressor','LinearRegression','Ridge','Lasso', 'SGDRegressor','RandomForestRegressor','ExtraTreesRegressor',
    'DecisionTreeRegressor','XGBRegressor','LGBMRegressor','GradientBoostingRegressor'
]
temp2 = [
    KNeighborsRegressor(),LinearRegression(),Ridge(),Lasso(), SGDRegressor(),RandomForestRegressor(),ExtraTreesRegressor(),
    DecisionTreeRegressor(),XGBRegressor(),LGBMRegressor(),GradientBoostingRegressor()
]
models = tuple( zip(temp1,temp2))
models

(('KNeighborsRegressor', KNeighborsRegressor()),
 ('LinearRegression', LinearRegression()),
 ('Ridge', Ridge()),
 ('Lasso', Lasso()),
 ('SGDRegressor', SGDRegressor()),
 ('RandomForestRegressor', RandomForestRegressor()),
 ('ExtraTreesRegressor', ExtraTreesRegressor()),
 ('DecisionTreeRegressor', DecisionTreeRegressor()),
 ('XGBRegressor',
  XGBRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=None, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               n_est

In [261]:
import numpy as np
for name,model in models:
  score  = cross_validate(model,x_train_scaled,y_train)
  print(f'{name} : {np.mean(score["test_score"])}')

KNeighborsRegressor : 0.6693598286091967
LinearRegression : 0.5813469866793826
Ridge : 0.5813659358733126
Lasso : 0.5816589670822431
SGDRegressor : 0.5808138774120747
RandomForestRegressor : 0.7662345998105494
ExtraTreesRegressor : 0.7831870456316177
DecisionTreeRegressor : 0.5343022362440903
XGBRegressor : 0.7609704731011966
LGBMRegressor : 0.758873200528528
GradientBoostingRegressor : 0.757570066744976


In [262]:
# 비선형으로 예측하기 2차원
import numpy as np
pf = PolynomialFeatures(degree=2)
x_train_poly = pf.fit_transform(x_train)
ss = StandardScaler()
x_train_poly_scaled = ss.fit_transform(x_train_poly)
for name,model in models:
  score  = cross_validate(model,x_train_poly_scaled,y_train)
  print(f'{name} : {np.mean(score["test_score"])}')

KNeighborsRegressor : 0.657944573744866
LinearRegression : 0.6274844004727933
Ridge : 0.6312565126836854
Lasso : 0.6278180750181435
SGDRegressor : 0.6231305647394193
RandomForestRegressor : 0.7569167214629445
ExtraTreesRegressor : 0.7899400804985611
DecisionTreeRegressor : 0.5013134189196495
XGBRegressor : 0.7473853054043834
LGBMRegressor : 0.7737105136641451
GradientBoostingRegressor : 0.7529022215932455


In [263]:
# 비선형으로 예측하기 3차원
import numpy as np
pf = PolynomialFeatures(degree=3)
x_train_poly = pf.fit_transform(x_train)
ss = StandardScaler()
x_train_poly_scaled = ss.fit_transform(x_train_poly)
for name,model in models:
  score  = cross_validate(model,x_train_poly_scaled,y_train)
  print(f'{name} : {np.mean(score["test_score"])}')

KNeighborsRegressor : 0.6660609700922837
LinearRegression : 0.5712343348279636
Ridge : 0.6690737877062002
Lasso : 0.6347010361655256
SGDRegressor : -4.102969595318666e+17
RandomForestRegressor : 0.7509137953727328
ExtraTreesRegressor : 0.7835086137987355
DecisionTreeRegressor : 0.5000398048116784
XGBRegressor : 0.762766466760462
LGBMRegressor : 0.7668470772970029
GradientBoostingRegressor : 0.7637137904846665


In [264]:
# 결정트리로 중요 피처만 골라서 재 학습
dt = DecisionTreeRegressor()
dt.fit(x_train,y_train)
np.sort(dt.feature_importances_), np.argsort(dt.feature_importances_)
feature_dict = dict(zip(x_train.columns, dt.feature_importances_))
sorted_items = sorted(feature_dict.items(), key=lambda x : x[1],reverse=True)
print(sorted_items)
top4cols = [a for a,b in sorted_items[:2]]
x_train_top4 = x_train.loc[:,top4cols]
# x_train_top4.head()
# 스케일링
from sklearn.preprocessing import RobustScaler
ss = RobustScaler()
x_train_top4_scaled = ss.fit_transform(x_train_top4)

for name,model in models:
  score  = cross_validate(model,x_train_top4_scaled,y_train)
  print(f'{name} : {np.mean(score["test_score"])}')

[('hour', 0.5894391873998772), ('hour_bef_temperature', 0.17809589122948388), ('hour_bef_humidity', 0.046656404587064354), ('hour_bef_ozone', 0.04537132610534237), ('hour_bef_windspeed', 0.033228931953904146), ('hour_bef_precipitation', 0.029400572857514372), ('hour_bef_pm10', 0.028187598057183995), ('hour_bef_pm2.5', 0.027521153427305725), ('hour_bef_visibility', 0.02209893438232383)]
KNeighborsRegressor : 0.6574307949617426
LinearRegression : 0.5412496511646111
Ridge : 0.5412582330993121
Lasso : 0.5408335273031938
SGDRegressor : 0.5412839841891532
RandomForestRegressor : 0.66611256956365
ExtraTreesRegressor : 0.6042327675291114
DecisionTreeRegressor : 0.5498282276052132
XGBRegressor : 0.6364495127595005
LGBMRegressor : 0.6997137574661194
GradientBoostingRegressor : 0.7167434033559557


In [265]:
# 피처의 개수가 많아야 성능이 높아짐.... 
from sklearn.decomposition import PCA
ss = StandardScaler()
pf = PolynomialFeatures(degree=3)
pca = PCA(n_components=0.5)
x_train_pca = pca.fit_transform(x_train)
x_train_pca_scaled = ss.fit_transform(x_train_pca)
x_train_pca_scaled_poly = pf.fit_transform(x_train_pca_scaled)

for name,model in models:
  score  = cross_validate(model,x_train_pca_scaled_poly,y_train)
  print(f'{name} : {np.mean(score["test_score"])}')



KNeighborsRegressor : -0.061406314322779744
LinearRegression : 0.07541234633768286
Ridge : 0.0754887937194777
Lasso : 0.07518388196740595
SGDRegressor : 0.07529968463541928
RandomForestRegressor : -0.29729221582444076
ExtraTreesRegressor : -0.4688510073887828
DecisionTreeRegressor : -0.7641245671918997
XGBRegressor : -0.21695443592993535
LGBMRegressor : 0.019402807973451463
GradientBoostingRegressor : 0.06578967390908812


In [266]:
# 가장 성능이 좋은 케이스
# 비선형으로 예측하기 2차원
import numpy as np
pf = PolynomialFeatures(degree=2)
pf.fit(x_train)
x_train_poly = pf.transform(x_train)
x_test_poly = pf.transform(x_test)
test_poly = pf.transform(test)

ss = StandardScaler()
ss.fit(x_train_poly)
x_train_poly_scaled = ss.transform(x_train_poly)
x_test_poly_scaled = ss.transform(x_test_poly)
test_poly_scaled = ss.transform(test_poly)

for name,model in models:
  score  = cross_validate(model,x_train_poly_scaled,y_train)
  print(f'{name} : {np.mean(score["test_score"])}')

KNeighborsRegressor : 0.657944573744866
LinearRegression : 0.6274844004727933
Ridge : 0.6312565126836854
Lasso : 0.6278180750181435
SGDRegressor : 0.6181780511218096
RandomForestRegressor : 0.7606919887815197
ExtraTreesRegressor : 0.7860921043322604
DecisionTreeRegressor : 0.49536807414923956
XGBRegressor : 0.7473853054043834
LGBMRegressor : 0.7737105136641451
GradientBoostingRegressor : 0.7534915516550296


In [267]:
# ExtraTreesRegressor 로 하이퍼 파라메터 수행하기
from sklearn.model_selection import RandomizedSearchCV
# 튜닝할 하이퍼파라미터 범위 설정
param_dist = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]    
}
model = ExtraTreesRegressor()
random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=10, cv=5, random_state=42)
random_search.fit(x_train_poly_scaled, y_train)
# 최적의 하이퍼파라미터와 성능 출력
print("Best Hyperparameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Hyperparameters: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': None}
Best Score: 0.7877281250901964


In [268]:
best_model = random_search.best_estimator_
best_model.score(x_test_poly_scaled,y_test) 

0.8183608988967839

In [269]:
x_test_poly_scaled.shape

(292, 55)

In [270]:
# 변수의 중요도 체크 : 주어진데이터를 랜던하게 학습해서 중요한 피처를 찾아줌
from sklearn.inspection import permutation_importance
# 변수 중요도 측정
x_train_poly_scaled.shape, y_train.shape
results = permutation_importance(best_model, x_train_poly_scaled, y_train, scoring='r2')
# 중요도 결과 출력
importance = results.importances_mean
np.argsort(importance)[::-1][:45]

# 튜닝할 하이퍼파라미터 범위 설정
param_dist = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]    
}
model2 = ExtraTreesRegressor()
random_search = RandomizedSearchCV(model2, param_distributions=param_dist, n_iter=10, cv=5, random_state=42)
random_search.fit(x_train_poly_scaled[:,np.argsort(importance)[::-1][:45]], y_train)
# 최적의 하이퍼파라미터와 성능 출력
print("Best Hyperparameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

best_model_test = random_search.best_estimator_
best_model_test.score(x_test_poly_scaled[:,np.argsort(importance)[::-1][:45]],y_test) 
# x_train_poly_scaled.shape[:,np.argsort(importance)[::-1][:25]]


Best Hyperparameters: {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': None}
Best Score: 0.7890784979340711


0.8109071023824004

## 학습 된 모델로 예측 데이터 생성

In [272]:
pred2 = best_model_test.predict(x_test_poly_scaled[:,np.argsort(importance)[::-1][:45]])
pred2.shape, x_test_poly_scaled.shape

((292,), (292, 55))

In [273]:
pred = best_model.predict(test_poly_scaled)
pred.shape,test_poly_scaled.shape

((715,), (715, 55))

현재까지 최고 성능요인 정리
  - 모델 : ExtraTreesRegressor
  - 전처리 : na는 중위수로 대체(id 컬럼 제거)
  - 2차원으로 차수증가
  - 스케일링(Standard)
  - 하이퍼 파라메터 튜닝
    - Best Hyperparameters: {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': None} 

In [313]:
train = pd.read_csv('train.csv') 
test = pd.read_csv('test.csv')
train = train.drop(['count'],axis = 1)
merge_df = pd.concat([train,test])
merge_df.mean()

id                        1089.779209
hour                        11.486661
hour_bef_temperature        18.870244
hour_bef_precipitation       0.038231
hour_bef_windspeed           2.449168
hour_bef_humidity           53.690465
hour_bef_visibility       1390.179641
hour_bef_ozone               0.039824
hour_bef_pm10               50.465559
hour_bef_pm2.5              28.517071
dtype: float64

In [314]:
train = pd.read_csv('train.csv') 
test = pd.read_csv('test.csv')
# 중위수로 결측치 처리하기
# train = train.fillna(train.median())
# test = test.fillna(test.median())

train = train.fillna(merge_df.mean())
test = test.fillna(merge_df.mean())

test = test.drop(['id'],axis=1)
train = train.drop(['id'],axis = 1)
# 섞기
arr = np.arange(len(train))
np.random.shuffle(arr)
train = train.iloc[arr,:]

train_x = train.drop(['count'],axis = 1)
train_y = train['count']


# 데이터 스케일링
from sklearn.preprocessing import RobustScaler
import numpy as np

pf = PolynomialFeatures(degree=2)
pf.fit(train_x)
train_x_poly = pf.transform(train_x)
test_poly = pf.transform(test)

ss = RobustScaler()
ss.fit(train_x_poly)
train_x_poly_scaled = ss.transform(train_x_poly)
test_poly_scaled = ss.transform(test_poly)

In [316]:
model = ExtraTreesRegressor(n_estimators= 100, min_samples_split= 2, min_samples_leaf= 2, max_depth= None )
model.fit(train_x_poly_scaled,train_y)
model.score(train_x_poly_scaled,train_y)
predict = model.predict(test_poly_scaled)

# RMSE 구하기
from sklearn.metrics import mean_squared_error
# MSE를 계산합니다.
mse = mean_squared_error(train_y,model.predict(train_x_poly_scaled))
# RMSE를 계산합니다.
rmse = np.sqrt(mse)
print(f'RMSE : {rmse}')

submission = pd.read_csv('submission.csv')
submission['count'] = pred
submission.to_csv('베이스라인.csv',index = False)

RMSE : 6.6332756125914685


변수의 중요도 체크해서 특정 feature 선택해 보기

In [307]:
from sklearn.inspection import permutation_importance
# 변수 중요도 측정
x_train_poly_scaled.shape, y_train.shape
results = permutation_importance(model, train_x_poly_scaled, train_y, scoring='r2')
# 중요도 결과 출력
importance = results.importances_mean
importance_index = np.argsort(importance)[::-1][:25]
print(train_x_poly_scaled[:,importance_index].shape, train_x_poly_scaled.shape)
new_data_train_x = train_x_poly_scaled[:,importance_index]
new_data_test_x = test_poly_scaled[:,importance_index]
#  재 학습
new_model = ExtraTreesRegressor(n_estimators= 100, min_samples_split= 2, min_samples_leaf= 2, max_depth= None )
new_model.fit(new_data_train_x,train_y)
#예측
predict = new_model.predict(new_data_test_x)
#결과저장
submission = pd.read_csv('submission.csv')
submission['count'] = pred
submission.to_csv('베이스라인.csv',index = False)

(1459, 25) (1459, 55)


## 제출파일 생성

In [ ]:
submission = pd.read_csv('submission.csv')
submission2 = pd.read_csv('submission.csv')

In [ ]:
submission['count'] = pred
submission2['count'] = pred2

In [ ]:
submission.to_csv('베이스라인.csv',index = False)
submission2.to_csv('베이스라인2.csv',index = False)